In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!pip install https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp37-cp37m-linux_x86_64.whl
!pip install torch==1.6

In [ ]:
# VERSION = "20200318"
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:
import numpy as np 
import pandas as pd
from PIL import Image
import os

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Subset
import cv2
import time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
!ls ../input/d/smc219/data-hotel-model

In [ ]:
import cv2

In [ ]:
BATCH = 1000
EPOCHS = 100


LR = 0.01
IM_SIZE = 32
DEVICE = xm.xla_device()
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

path = '../input/hotelid2021fgvc8-728x1024/hotel-id-2021-fgvc8_728x1024/'
val_path = '../input/hotel-id-2021-fgvc8/'
TRAIN_DIR = path + 'train_images/'
TEST_DIR = path + 'test_images/'
print(DEVICE)

In [ ]:
# switch to val path
# path = val_path

In [ ]:
train_df = pd.read_csv(path+'train.csv')
train_df
path = val_path

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_df['chain'])
train_df['label'] = le.transform(train_df['chain'])
train_df

In [ ]:
# for Inference

class_map = dict(sorted(train_df[['label', 'chain']].values.tolist()))
class_map

In [ ]:
Transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
class GetData(Dataset):
    def __init__(self, Dir, FNames, Labels, Transform):
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels         
        
    def __len__(self): 
        return len(self.fnames)

    def __getitem__(self, index):       
#         x = Image.open(os.path.join(self.dir, self.fnames[index]))
        x = np.array(cv2.imread(os.path.join(self.dir, self.fnames[index]), cv2.IMREAD_COLOR))
        if "train" in self.dir:             
            return self.transform(x), self.labels[index]
        elif "test" in self.dir:            
            return self.transform(x), self.fnames[index]

In [ ]:
train_df['chain_image'] = train_df['chain'].astype(str) + '/' + train_df['image']
train_df

In [ ]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [ ]:
tr_df = train_df[:10000]

In [ ]:


train_df['chain_image'] = train_df['chain'].astype(str) + '/' + train_df['image']
tr_df = train_df

print(len(tr_df))
X_Train, Y_Train = tr_df['chain_image'].values, tr_df['label'].values

# X_Train = train_df['chain_image'].values
# Y_Train = train_df['label'].values

trainset = GetData(TRAIN_DIR, X_Train, Y_Train, Transform)
totalset = train_val_dataset(trainset)
train_loader = DataLoader(totalset['train'], batch_size=BATCH, shuffle=True)
val_loader = DataLoader(totalset['val'], batch_size=BATCH, shuffle=False)

In [ ]:
# next(iter(val_loader))[0].shape

In [ ]:
train_df[train_df['label'] == 16]

In [ ]:
NUM_CL = len(train_df['chain'].value_counts())
NUM_CL

In [ ]:

model = torchvision.models.resnet50()
model.fc = nn.Linear(2048, NUM_CL, bias=True)
# state_dict = xser.load('../input/d/smc219/data-hotel-model/22.pth')
model.load_state_dict(torch.load('../input/d/smc219/data-hotel-model/22.pth', map_location=torch.device('cpu')))
# model.load_state_dict(state_dict)
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
print(model)

In [ ]:
# s_ls = []
# correct = 0
# total = 0
# with torch.no_grad():
#     model.eval()
#     for image, fname in val_loader: 
       
#         image = image.to(DEVICE)
    
#         logits = model(image)    
#         if total == 0: print(logits)
        
#         ps = torch.exp(logits)        
#         _, top_class = ps.topk(1, dim=1)
#         if total == 0: print(ps)
#         total += image.size()[0]
#         for i in range(len(top_class)):
#             pred = top_class[i]
#             correct += (pred == fname[i]).sum().float()
            
            
# print(correct / 250 * 100)

In [ ]:
# print(correct * 100 / total)

In [ ]:
# print(correct)
# print(total)
# print(output.shape)
# print(labels.shape)
# _, predicted = torch.max(outputs.data, 1)
# print(predicted.shape)

In [ ]:
model.train()

In [ ]:

model.to(DEVICE)
for epoch in range(EPOCHS):
    tr_loss = 0.0
    
    start = time.time()
    model = model.train()

    for i, (images, labels) in enumerate(train_loader):   
        start = time.time()
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)       
        logits = model(images.float())       
        loss = criterion(logits, labels)
        optimizer.zero_grad()

            
        optimizer.step()
        
        
        
        tr_loss += loss.detach().item()
        print("{} time : {}".format(i, time.time() - start))
    torch.save(model.state_dict(), './2021_0519' + str(epoch) + '.pth')
    model.eval()
    print('Epoch: %d | Loss: %.4f'%(epoch, tr_loss / i))

In [ ]:
torch.save(model.state_dict(), './2021_0519_fin.pth')